# 3-digit addition learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from keras.layers import Dense, Bidirectional, LSTM, RepeatVector
from keras.models import Sequential

from utils import CharacterTable, vectorization, train_val_split, model_inference, fitting_visualize

# Parameters for the model and dataset.
training_size = 50000
digits = 3
reverse = False

# Maximum length of 'int + int'
maxlen = digits + 1 + digits

## Data Generation

In [ ]:
# The resulting number is 3 or 4 digits and sometimes includes ' ', so the string of ' ' is also considered.
chars = "0123456789+ "
ctable = CharacterTable(chars)

In [ ]:
questions = []
expected = []
seen = set()

print("Generating data...")

while len(questions) < training_size:
  # Randomly generate a and b from 'a+b'
  a, b = np.random.randint(1,1000), np.random.randint(1,1000)

  # Skip when duplicate a+b is calculated because a+b = b+a
  key = tuple(sorted([a, b]))
  if key in seen:
    continue
  else:
    seen.add(key)

  # If there is an extra length according to maxlen, padding is performed by adding a trailing space
  q = "{}+{}".format(a, b)
  query = q + " " * (maxlen - len(q))
  
  # The remaining part of the maximum length of the correct answer that can be printed is added as a space
  ans = str(a + b)
  ans += " " * (digits + 1 - len(ans))

  if reverse:  
    query = query[::-1]

  questions.append(query)
  expected.append(ans)
  
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


## Data Vectorization

In [ ]:
x, y = vectorization(questions, expected, chars, maxlen, digits+1, ctable)

x_train, x_val, y_train, y_val = train_val_split(x,y)

Training Data:
(45000, 7, 12)
(45000, 4, 12) 

Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [ ]:
print('Question:', questions[53], '\n')
print('Answer:', expected[53], '\n')

print("chars location", sorted(set(chars)),'\n')

print('Encoded Question:\n\n', x_train[53], '\n')
print('Encoded Answer:\n\n', y_train[53])

Question: 728+786 

Answer: 1514 

chars location [' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 

Encoded Question:

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]] 

Encoded Answer:

 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


## Modeling

In [ ]:
# Build LSTM Sequence Model
def model_basic(num_layers):
  model=Sequential()
  model.add(LSTM(128,input_shape=(maxlen, len(chars))))
  model.add(RepeatVector(digits+1)) # convert because target value has a row of 4
  for _ in range(num_layers):
    model.add(LSTM(128, return_sequences=True))
  model.add(Dense(len(chars),activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

  return model

In [ ]:
model= model_basic(1)

epochs=35
batch_size=32

fitting_visualize(x_train, y_train, x_val, y_val, model, epochs, batch_size, ctable)


Iteration 1
1407/1407 [==============================] - 41s 6ms/step - loss: 1.9099 - acc: 0.3167 - val_loss: 1.4783 - val_acc: 0.4503
Q 150+977 T 1127 x 1100
Q 378+563 T 941  x 900 
Q 151+557 T 708  x 680 
Q 742+498 T 1240 x 1200
Q 120+8   T 128  x 388 
Q 839+630 T 1469 x 1400
Q 766+859 T 1625 x 1600
Q 573+966 T 1539 x 1500
Q 873+880 T 1753 x 1700
Q 92+100  T 192  x 410 

Iteration 2
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4050 - acc: 0.4738 - val_loss: 1.3191 - val_acc: 0.5041
Q 67+613  T 680  x 700 
Q 650+34  T 684  x 700 
Q 230+848 T 1078 x 1070
Q 700+541 T 1241 x 1299
Q 126+305 T 431  x 450 
Q 418+729 T 1147 x 1157
Q 784+621 T 1405 x 1310
Q 638+345 T 983  x 900 
Q 63+614  T 677  x 700 
Q 168+249 T 417  x 450 

Iteration 3
1407/1407 [==============================] - 8s 6ms/step - loss: 1.2304 - acc: 0.5420 - val_loss: 1.0836 - val_acc: 0.6052
Q 373+311 T 684  x 688 
Q 198+553 T 751  x 747 
Q 927+5   T 932  x 17  
Q 834+347 T 1181 x 1187
Q 374+247 T 621